In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from custom_models import cnn
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import time
import os
from PIL import Image
from torchvision.transforms import v2
from tempfile import TemporaryDirectory
import shutil
import random

cudnn.benchmark = True
plt.ion()   # interactive mode

#### Data Transformations

In [3]:
# data transformations to loop through
minimal_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

basic_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.RandomHorizontalFlip(p=0.5),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

auto_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.AutoAugment(policy=v2.AutoAugmentPolicy.IMAGENET),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

#### Create Training Set with Real/Synthetic Images

In [4]:
def makeSyntheticTrain(train_directory, synthetic_train_directory, train_percentage, synthetic_percentage):

    # Remove any existing images in directory
    try:
        shutil.rmtree(synthetic_train_directory)
    except:
        print("directory does not exist")

    # Loop through subfolders, generate synthetic images
    subfolders = [f for f in os.listdir(train_directory)]

    for s in subfolders:
        # for each subfolder in the train directory, make the same in the synthetic train directory
        os.makedirs(f"{synthetic_train_directory}/{s}", exist_ok=True)
        
        # get a random sample from each subfolder
        subfolder_path = f"{train_directory}/{s}"
        files = os.listdir(subfolder_path)
        sample_files = random.sample(files, round(len(files)*train_percentage))
        
        # create synthetic sample based on sampled original images
        synthetic_subfolder_path = subfolder_path.replace('train','synthetic')
        synthetic_files = [f for f in os.listdir(synthetic_subfolder_path) if int(f.replace('.png','').split('_')[1]) in [int(f.replace('.png','').split('_')[1]) for f in sample_files]]
        synthetic_sample_files = random.sample(synthetic_files, round(len(files)*synthetic_percentage))
        
        # Move sample files to synthetic directory
        for f in sample_files:
            
            image_path = f"{subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

        # Move synthetic sample files to synthetic directory
        for f in synthetic_sample_files:

            image_path = f"{synthetic_subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

#### Read Data

In [5]:
def get_data(data_dir, data_sets, data_transforms):
    
    image_datasets = {
        x: datasets.ImageFolder(
            os.path.join(data_dir, x),
            data_transforms[x]
        )
        for x in data_sets
    }

    dataloaders = {
        x: DataLoader(
            image_datasets[x], 
            batch_size=4,
            shuffle=True, 
            num_workers=4
        )
        for x in data_sets
    }

    dataset_sizes = {
        x: len(image_datasets[x]) 
        for x in data_sets
    }

    class_names = image_datasets['synthetic_train'].classes

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    return image_datasets, dataloaders, dataset_sizes, class_names, device

#### Train Model

In [6]:
def train_model(model, criterion, optimizer, dataloaders, dataset_sizes, num_epochs=10):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        
        best_test_loss = float('inf')
        patience = 2  # Number of epochs to wait for improvement before stopping
        test_losses = []
        train_losses = []
        test_acc = []
        train_acc = []
        patience_counter = 0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['synthetic_train', 'test']:
                if phase == 'synthetic_train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'synthetic_train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'synthetic_train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                
                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double().item() / dataset_sizes[phase]

                if phase =='synthetic_train':
                    train_losses.append(epoch_loss)
                    train_acc.append(epoch_acc)
                else:
                    test_losses.append(epoch_loss)
                    test_acc.append(epoch_acc)
                    
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'test' and epoch_loss <= best_test_loss:
                    best_test_loss = epoch_loss
                    patience_counter = 0  # Reset counter
                    torch.save(model.state_dict(), best_model_params_path)
                elif phase == 'test' and epoch_loss > best_test_loss:
                    patience_counter += 1
                
                # Early stopping check
                #if patience_counter >= patience:
                #    print("Stopping early due to no improvement in validation loss.")
                #    break

        # store results in dataframe
        dat = {
            "epoch": range(len(test_losses)),
            "test_losses": test_losses,
            "train_losses": train_losses,
            "test_accuracies": test_acc,
            "train_accuracies": train_acc
        }

        result = pd.DataFrame(data=dat)
        print()
        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best test Loss: {best_test_loss:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return result, model

#### Loop through different training scenarios

In [7]:
num_epochs = 10
num_sims = 10
train_percs = [1.0, 0.8, 0.8, 0.6, 0.6, 0.4, 0.4]
synth_percs = [0.0, 0.2, 0.0, 0.4, 0.0, 0.6, 0.0]
transforms = {
    'minimal': minimal_transforms,
    'basic': basic_transforms,
    'auto': auto_transforms
}

for train_percentage, synthetic_percentage in zip(train_percs, synth_percs):
    for active_transform in transforms.keys():
        df_all_results = pd.DataFrame()
        for n in range(num_sims):
    
            # make the synthetic training dataset
            makeSyntheticTrain(
                train_directory='../data/alzheimer_mri/train',
                synthetic_train_directory='../data/alzheimer_mri/synthetic_train', 
                train_percentage=train_percentage, 
                synthetic_percentage=synthetic_percentage
            )

            # get and load datasets
            data_dir = '../data/alzheimer_mri'
            data_sets = ['synthetic_train','test']
            image_datasets, dataloaders, dataset_sizes, class_names, device = get_data(
                data_dir=data_dir, 
                data_sets=data_sets, 
                data_transforms=transforms.get(active_transform)
            )

            # instantiate model
            model = cnn.CNN(in_channels=1, num_classes=4)

            # Set the size of each output sample to nn.Linear(num_ftrs, len(class_names))
            #num_ftrs = 4 #model.fc.in_features
            #model.fc = nn.Linear(num_ftrs, len(class_names))
            model = model.to(device)

            # train the model
            df_results,_ = model = train_model(
                model=model, 
                criterion = nn.CrossEntropyLoss(),
                optimizer = optim.Adam(model.parameters(), lr=0.001), 
                dataloaders=dataloaders,
                dataset_sizes=dataset_sizes,
                num_epochs=num_epochs
            )

            df_results['train_percentage'] = train_percentage
            df_results['synth_percentage'] = synthetic_percentage
            df_results['transform'] = active_transform
            df_results['sim_num'] = n
            df_results['category'] = df_results.apply(lambda row: row['transform']+'_'+str(row['train_percentage'])+'_'+str(row['synth_percentage']), axis=1)
            df_all_results = pd.concat([df_all_results, df_results],ignore_index=True)
        df_all_results.to_csv('../results/results_cnn_'+active_transform+'_'+str(train_percentage)+'_'+str(synthetic_percentage)+'.csv')

/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 0.8892 Acc: 0.5756
test Loss: 0.8768 Acc: 0.6273
Epoch 1/9
----------
synthetic_train Loss: 0.4115 Acc: 0.8307
test Loss: 0.2435 Acc: 0.9203
Epoch 2/9
----------
synthetic_train Loss: 0.1210 Acc: 0.9568
test Loss: 0.3129 Acc: 0.9070
Epoch 3/9
----------
synthetic_train Loss: 0.0561 Acc: 0.9805
test Loss: 0.2670 Acc: 0.9187
Epoch 4/9
----------
synthetic_train Loss: 0.0467 Acc: 0.9842
test Loss: 0.1067 Acc: 0.9641
Epoch 5/9
----------
synthetic_train Loss: 0.0505 Acc: 0.9812
test Loss: 0.1659 Acc: 0.9500
Epoch 6/9
----------
synthetic_train Loss: 0.0196 Acc: 0.9938
test Loss: 0.1758 Acc: 0.9484
Epoch 7/9
----------
synthetic_train Loss: 0.0299 Acc: 0.9906
test Loss: 0.1299 Acc: 0.9695
Epoch 8/9
----------
synthetic_train Loss: 0.0326 Acc: 0.9902
test Loss: 0.1948 Acc: 0.9563
Epoch 9/9
----------
synthetic_train Loss: 0.0319 Acc: 0.9920
test Loss: 0.1056 Acc: 0.9734

Training complete in 3m 17s
Best test Loss: 0.105595


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 0.8341 Acc: 0.6119
test Loss: 0.5454 Acc: 0.7805
Epoch 1/9
----------
synthetic_train Loss: 0.2707 Acc: 0.8949
test Loss: 0.2044 Acc: 0.9344
Epoch 2/9
----------
synthetic_train Loss: 0.0763 Acc: 0.9725
test Loss: 0.1684 Acc: 0.9539
Epoch 3/9
----------
synthetic_train Loss: 0.0633 Acc: 0.9799
test Loss: 0.1484 Acc: 0.9641
Epoch 4/9
----------
synthetic_train Loss: 0.0253 Acc: 0.9910
test Loss: 0.1501 Acc: 0.9664
Epoch 5/9
----------
synthetic_train Loss: 0.0351 Acc: 0.9855
test Loss: 0.1484 Acc: 0.9586
Epoch 6/9
----------
synthetic_train Loss: 0.0287 Acc: 0.9912
test Loss: 0.1178 Acc: 0.9641
Epoch 7/9
----------
synthetic_train Loss: 0.0196 Acc: 0.9938
test Loss: 0.1342 Acc: 0.9602
Epoch 8/9
----------
synthetic_train Loss: 0.0271 Acc: 0.9928
test Loss: 0.1446 Acc: 0.9672
Epoch 9/9
----------
synthetic_train Loss: 0.0153 Acc: 0.9949
test Loss: 0.1294 Acc: 0.9711

Training complete in 3m 17s
Best test Loss: 0.117765


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

Epoch 0/9
----------
synthetic_train Loss: 0.9712 Acc: 0.5240
test Loss: 0.8958 Acc: 0.5258
Epoch 1/9
----------
synthetic_train Loss: 0.8284 Acc: 0.6023
test Loss: 0.7788 Acc: 0.6328
Epoch 2/9
----------
synthetic_train Loss: 0.6198 Acc: 0.7285
test Loss: 0.6446 Acc: 0.6953
Epoch 3/9
----------
synthetic_train Loss: 0.4112 Acc: 0.8391
test Loss: 0.5302 Acc: 0.7891
Epoch 4/9
----------
synthetic_train Loss: 0.2620 Acc: 0.8977
test Loss: 0.3418 Acc: 0.8836
Epoch 5/9
----------
synthetic_train Loss: 0.1879 Acc: 0.9309
test Loss: 0.2830 Acc: 0.9031
Epoch 6/9
----------
synthetic_train Loss: 0.1331 Acc: 0.9547
test Loss: 0.4018 Acc: 0.8539
Epoch 7/9
----------
synthetic_train Loss: 0.1108 Acc: 0.9623
test Loss: 0.2738 Acc: 0.9078
Epoch 8/9
----------
synthetic_train Loss: 0.0837 Acc: 0.9734
test Loss: 0.1855 Acc: 0.9437
Epoch 9/9
----------
synthetic_train Loss: 0.0689 Acc: 0.9764
test Loss: 0.3363 Acc: 0.9078

Training complete in 3m 30s
Best test Loss: 0.185454


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0589 Acc: 0.4965
test Loss: 1.0255 Acc: 0.4766
Epoch 1/9
----------
synthetic_train Loss: 1.0310 Acc: 0.5051
test Loss: 0.9321 Acc: 0.5523
Epoch 2/9
----------
synthetic_train Loss: 0.9878 Acc: 0.5236
test Loss: 0.8832 Acc: 0.5609
Epoch 3/9
----------
synthetic_train Loss: 0.9460 Acc: 0.5484
test Loss: 0.7923 Acc: 0.6297
Epoch 4/9
----------
synthetic_train Loss: 0.8853 Acc: 0.5775
test Loss: 0.6947 Acc: 0.6984
Epoch 5/9
----------
synthetic_train Loss: 0.8437 Acc: 0.6111
test Loss: 0.6077 Acc: 0.7508
Epoch 6/9
----------
synthetic_train Loss: 0.7900 Acc: 0.6309
test Loss: 0.5308 Acc: 0.7641
Epoch 7/9
----------
synthetic_train Loss: 0.7365 Acc: 0.6637
test Loss: 0.4371 Acc: 0.8187
Epoch 8/9
----------
synthetic_train Loss: 0.6853 Acc: 0.6939
test Loss: 0.3640 Acc: 0.8648
Epoch 9/9
----------
synthetic_train Loss: 0.6398 Acc: 0.7104
test Loss: 0.4234 Acc: 0.8477

Training complete in 4m 4s
Best test Loss: 0.364001


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0514 Acc: 0.4926
test Loss: 1.0028 Acc: 0.5000
Epoch 1/9
----------
synthetic_train Loss: 1.0174 Acc: 0.5111
test Loss: 0.9038 Acc: 0.5320
Epoch 2/9
----------
synthetic_train Loss: 0.9888 Acc: 0.5201
test Loss: 0.8667 Acc: 0.5883
Epoch 3/9
----------
synthetic_train Loss: 0.9540 Acc: 0.5424
test Loss: 0.8311 Acc: 0.6125
Epoch 4/9
----------
synthetic_train Loss: 0.9017 Acc: 0.5732
test Loss: 0.6860 Acc: 0.6781
Epoch 5/9
----------
synthetic_train Loss: 0.8286 Acc: 0.6180
test Loss: 0.5525 Acc: 0.7562
Epoch 6/9
----------
synthetic_train Loss: 0.7730 Acc: 0.6477
test Loss: 0.5135 Acc: 0.7945
Epoch 7/9
----------
